In [1]:
import multiprocessing
import pdfplumber
import os
import time

In [2]:
def get_ranges(num_items, n_process):

    if(n_process == 1):
        return 0, (num_items - 1)

    total_len = num_items
    num_process = n_process
    lower = []
    upper = []
    step = int(total_len/num_process)

    for k in range(num_process):
        lower.append(0)
        upper.append(0)

    lower[0] = 0
    upper[0] = step

    i = 1
    j = 0
    while (i < num_process):
        upper[i]  = upper[j] + step
        lower[i]  = upper[j] +  1
        if(i%2 != 0):
            upper[i] = upper[i] + 1

        i = i + 1
        j = j + 1

    upper[n_process - 1] = num_items - 1

    return lower, upper

In [3]:
def read_pdf(files, i, results_process, file_path):
    """
    files: lista de arquivos a serem processados
    i: id do processo
    results_process: dict que recebe a saída da função
    file_path: diretorio para o arquivo
    
    A ideia é fazer igual o pessoal do F03, extrair todas as informações e
    retornar apenas as caracterizações.
    """
    pdfs = {}
    for file in files:
        pages = []
        tables = []
        pdf = pdfplumber.open(file_path+file)
        for page in pdf.pages:
            pages.append(page.extract_text())
            tables.append(page.extract_tables())
        #pdfs.append(pdf.pages[0].extract_text())
        pdfs[file] = [pages, tables]
        
    results_process[i] = pdfs

In [4]:
def preprocess_items(items, file_path,n_process=10):
    # It defines the ranges (of the items) the process will work on:
    process_ranges = get_ranges(len(items), n_process)
    if n_process == 1:
        process_ranges = [[process_ranges[0]], [process_ranges[1]]]

    print('Read ranges')
    print(process_ranges)

    manager = multiprocessing.Manager()
    results_process = manager.dict()
    jobs = []

    for i in range(n_process):
        lower = process_ranges[0][i]
        upper = process_ranges[1][i]
        items_process = items[lower:(upper + 1)]
        p = multiprocessing.Process(target=read_pdf,
        args = (items_process, i, results_process, file_path))
        jobs.append(p)
        p.start()

    del items

    for proc in jobs:
        proc.join()
        proc.close()

    items_descriptions = []
    return results_process
    for i in range(n_process):
        items_descriptions += results_process[i]

    return items_descriptions

In [5]:
file_path = "../data/288_licitacoes-pirapetinga/data/files/"
files = os.listdir(file_path)
print("Total ",len(files))
# seleciona apenas pdfs
files = [file for file in files if ".pdf" in file]
print("Apenas pdfs ",len(files))
# Seleciona amostra para testes
files = files[:50]
print(len(files))

Total  1113
Apenas pdfs  1007
50


In [35]:
# Multi threads
start = time.time()
results = preprocess_items(files, file_path, 10)
time.time() - start

Read ranges
([0, 6, 12, 17, 23, 28, 34, 39, 45, 50], [5, 11, 16, 22, 27, 33, 38, 44, 49, 49])


63.66626787185669

In [37]:
# Single thread
start = time.time()
results = preprocess_items(files, file_path, 1)
time.time() - start

Read ranges
[[0], [49]]


205.3191978931427